In [2]:
import ollama
import pandas as pd
from tqdm import tqdm
from grounded_sam.inference import grounded_segmentation
from grounded_sam.plot import plot_detections_plotly, plot_detections
from PIL import Image

/Users/kasparbeelen/anaconda3/envs/llm-segment/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
collection_df = pd.read_csv('./data/heritage_weaver_data.csv')
collection_df.head()

,Unnamed: 0,record_id,name,description,category,taxonomy,img_loc,img_name,img_path,downloaded,img_url,collection
0,0,co8413501,audio cassette,BBC Radio Manchester Audiopak.\n\nBroadcast ca...,Radio Communication,audio tape; sound recording; <information arte...,204/255/medium_cd0620_049_100527_2005_86_35_Pr...,204|255|medium_cd0620_049_100527_2005_86_35_Pr...,smg_imgs/204|255|medium_cd0620_049_100527_2005...,True,https://coimages.sciencemuseumgroup.org.uk/204...,smg
1,1,co8413501,audio cassette,BBC Radio Manchester Audiopak.\n\nBroadcast ca...,Radio Communication,audio tape; sound recording; <information arte...,477/975/medium_SMG00247371.jpg,477|975|medium_SMG00247371.jpg,smg_imgs/477|975|medium_SMG00247371.jpg,True,https://coimages.sciencemuseumgroup.org.uk/477...,smg
2,2,co5606,nuclear fuel,Fuel element (without fuel) from the Dounreay ...,Nuclear Energy,NaN,58/255/medium_1982_1712__0001_.jpg,58|255|medium_1982_1712__0001_.jpg,smg_imgs/58|255|medium_1982_1712__0001_.jpg,True,https://coimages.sciencemuseumgroup.org.uk/58/...,smg
3,3,co5606,nuclear fuel,Fuel element (without fuel) from the Dounreay ...,Nuclear Energy,NaN,58/256/medium_1982_1712__0002_.jpg,58|256|medium_1982_1712__0002_.jpg,smg_imgs/58|256|medium_1982_1712__0002_.jpg,True,https://coimages.sciencemuseumgroup.org.uk/58/...,smg
4,4,co5606,nuclear fuel,Fuel element (without fuel) from the Dounreay ...,Nuclear Energy,NaN,58/257/medium_1982_1712__0003_.jpg,58|257|medium_1982_1712__0003_.jpg,smg_imgs/58|257|medium_1982_1712__0003_.jpg,True,https://coimages.sciencemuseumgroup.org.uk/58/...,smg


In [4]:
df_reduced = collection_df.drop_duplicates(subset=['description','img_path'])
df_reduced.shape, collection_df.shape

((27690, 12), (28476, 12))

In [5]:
def get_completion_llava(system, prompt, img_path):
  stream = ollama.generate(
    model='llava',
    system=system,
    prompt=prompt,
    images=[img_path],
    stream=True,
    
  )
  completion = ''
  for chunk in stream:
    completion+=chunk['response']
  return completion

In [6]:
tqdm.pandas()
system_message = """
    You are a helpful AI that will assist me extracting keywords from image descriptions. 
    Keywords are returned as a comma separated list.

    """
user_message = """
    What do you see in this image? 
    Please extract two or three keywords based on what you see in the image. 
    Keywords should refer to objects and things that are visible in the image.
    Use only keywords from the description provided below between triple hashtags.
    Return keywords in a comma separated list, such as in the following example:
    keys, metal object, copper string.
    """

responses = []

def run_llava(x, system_message, user_message):
  try:
    prompt = user_message + f"\n\n###{x['description']}###"

    return get_completion_llava(system_message, prompt, x['img_path'])
  except Exception as e:
    return None




In [7]:
def segment_image(row,threshold = .5):
    image = Image.open(row['img_path'])
    image_x, image_y = image.size[0],image.size[1]
    image.resize((int(image_x/3), int(image_y/3)))
    labels = [x.strip().strip('#').strip('"').lower() for x in row['keywords'].split(',')]
    image_array, detections = grounded_segmentation(image, labels, threshold=threshold, polygon_refinement=True)
    return image_array, detections

In [8]:
df_sample = df_reduced.sample(n=10)
print('Extracting keywords and segmenting images...')
df_sample['keywords'] = df_sample.progress_apply(lambda x: run_llava(x, system_message, user_message), axis=1)
df_sample['segmentation'] = df_sample.progress_apply(lambda x: segment_image(x), axis=1)

Extracting keywords and segmenting images...


100%|██████████| 10/10 [05:11<00:00, 31.16s/it]


In [14]:
idx = 9
image_array, detections = df_sample.iloc[idx]['segmentation']
plot_detections_plotly(image_array, detections)


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'x: %{x}<br>y: %{y}<br>color: [%{z[0]}, %{z[1]}, %{z[2]}]<extra></extra>',
              'name': '0',
              'source': ('data:image/png;base64,iVBORw0K' ... 'G5qFMX9S6MBAAAAABJRU5ErkJggg=='),
              'type': 'image',
              'xaxis': 'x',
              'yaxis': 'y'},
             {'fill': 'toself',
              'line': {'color': 'tan', 'width': 2},
              'mode': 'lines',
              'name': 'electro-magnet.: 0.79',
              'type': 'scatter',
              'x': [98, 96, 96, 95, 95, 94, 94, 92, 91, 90, 79, 78, 77, 76, 76,
                    74, 68, 67, 67, 66, 66, 65, 65, 66, 66, 65, 65, 64, 64, 63, 63,
                    62, 62, 61, 62, 62, 61, 61, 60, 61, 61, 60, 60, 59, 59, 58, 58,
                    57, 57, 56, 56, 57, 57, 56, 56, 57, 58, 272, 273, 274, 299,
                    300, 301, 317, 318, 319, 320, 320, 321, 321, 320, 320, 319,
                    319, 318, 318, 317, 317, 316, 316, 315, 315, 314, 314, 313,
                    313, 312, 312, 311, 311, 312, 312, 313, 313, 314, 314, 316,
                    317, 327, 328, 364, 365, 366, 372, 373, 379, 380, 381, 381,
                    383, 383, 384, 384, 386, 386, 388, 390, 391, 392, 393, 399,
                    399, 400, 400, 401, 401, 400, 400, 399, 398, 397, 396, 394,
                    393, 392, 391, 387, 386, 382, 378, 378, 377, 377, 376, 376,
                    375, 367, 366, 365, 364, 352, 351, 348, 347, 340, 339, 321,
                    320, 319, 311, 309, 309, 308, 308, 307, 306, 303, 301, 301,
                    300, 261, 260, 259, 257, 257, 256, 256, 255, 255, 254, 250,
                    248, 104, 103, 101, 100, 98],
              'y': [135, 137, 140, 141, 149, 150, 155, 157, 157, 156, 156, 157,
                    157, 158, 160, 162, 162, 163, 174, 175, 203, 204, 206, 207,
                    209, 210, 235, 236, 256, 257, 276, 277, 299, 300, 301, 302,
                    303, 326, 327, 328, 329, 330, 352, 353, 374, 375, 400, 401,
                    419, 420, 422, 423, 424, 425, 445, 445, 446, 446, 445, 446,
                    446, 445, 446, 446, 445, 445, 444, 443, 442, 436, 435, 419,
                    418, 392, 391, 371, 370, 348, 347, 328, 327, 306, 305, 275,
                    274, 257, 256, 230, 229, 221, 220, 219, 218, 215, 214, 200,
                    198, 199, 199, 200, 200, 201, 200, 200, 201, 201, 200, 200,
                    199, 197, 194, 193, 190, 188, 187, 185, 185, 186, 186, 187,
                    187, 186, 185, 182, 181, 170, 169, 168, 168, 167, 167, 168,
                    168, 169, 169, 170, 170, 171, 167, 167, 168, 169, 170, 171,
                    174, 175, 175, 174, 174, 173, 173, 174, 174, 173, 173, 172,
                    172, 171, 172, 172, 170, 166, 165, 164, 164, 163, 163, 161,
                    158, 157, 157, 158, 158, 156, 149, 148, 138, 137, 136, 135,
                    135, 137, 137, 136, 136, 135, 135]},
             {'fill': 'toself',
              'line': {'color': 'burlywood', 'width': 2},
              'mode': 'lines',
              'name': 'roller escapement switch.: 0.79',
              'type': 'scatter',
              'x': [98, 96, 96, 95, 95, 94, 94, 92, 91, 90, 79, 78, 77, 76, 76,
                    74, 68, 67, 67, 66, 66, 65, 65, 66, 66, 65, 65, 64, 65, 65, 64,
                    64, 63, 63, 62, 62, 61, 62, 62, 61, 61, 60, 61, 61, 60, 60, 59,
                    59, 58, 58, 57, 57, 56, 56, 57, 57, 56, 56, 57, 58, 272, 273,
                    274, 299, 300, 301, 317, 318, 319, 320, 320, 321, 321, 320,
                    320, 319, 319, 318, 318, 317, 317, 316, 316, 315, 315, 314,
                    314, 313, 313, 312, 312, 311, 311, 312, 312, 313, 313, 314,
                    314, 316, 317, 327, 328, 364, 365, 366, 372, 373, 379, 380,
                    381, 381, 383, 383, 384, 384, 386, 386, 388, 390, 391, 392,
                    393, 399, 399, 400, 400, 401, 401, 400, 400, 399, 398, 397,
                    39